In [1]:
"""

PROD ALERT 2 CODE

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

keithreid@nhs.net

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the “Software”), to deal in the Software without restriction, including without limitation 
the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions 
of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED 
TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL 
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE.

"""

"""

From PROD-ALERT "1"
Institutions were categorised. “Small” were the continuous series of smallest institutions with “null” reports 
commensurate with size and having other monthly reports around 5 and 10. “Complete” institutions were bigger 
than “small” with one or less nulls. To interpolate single nulls among complete reporters, the last observation 
was carried forward, else next carried back (for month 1), both within the 12 months. Other institutions were 
neither small nor complete, and sub-categorised as “Partial”, “Joiner”, or “Null”.

By definition, “Joiners” consistently reported null then consistently reported restrained people – reporting none
then many. “Partial” reporter institutions had intermittent null but were not small enough to be in the 
continuous “small” series - reporting many people restrained then none then many, etc. “Null” report institutions
reported only null all year despite not being small – reporting none throughout while being the size of trusts 
reporting many. Small reporters with null restraints, and “joiner” or “partial” Bed-Day reporting style were 
categorised “null” – if bed counts were partial we did not trust restraint counts.
"""

"""
small for detentions has to be smaller than these two

SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
450 	510 	490 	480 	445 	450 	530 	525 	480 	510 	515 	505 
10  	10  	10  	10  	10  	10  	10  	5   	10  	10  	10  	5

EQUILIBRIUM HEALTHCARE
600 	650 	630 	650 	650 	500 	470 	450 	465 	435 	435 	400 
20    	20  	20  	20  	20  	15  	15  	15  	15  	15  	15  	10
"""
;

''

In [2]:
# libraries

import numpy  as np
import pandas as pd

In [3]:
# config
                                
pd.options.display.max_rows     = None
pd.options.display.max_columns  = None

"""
We are specificaly naming files in development but will glob them later
"""

def get_beds_files():
    beds_files = ["beds01_Sep_2021.csv", "beds02_Oct_2021.csv", "beds03_Nov_2021.csv",
                  "beds04_Dec_2021.csv", "beds05_Jan_2022.csv", "beds06_Feb_2022.csv",
                  "beds07_Mar_2022.csv", "beds08_Apr_2022.csv", "beds09_May_2022.csv",
                  "beds10_Jun_2022.csv", "beds11_Jul_2022.csv", "beds12_Aug_2022.csv"]
    return beds_files

def get_rstr_files():
    rstr_files = ["rstr01_Sep_2021.csv", "rstr02_Oct_2021.csv", "rstr03_Nov_2021.csv",
                  "rstr04_Dec_2021.csv", "rstr05_Jan_2022.csv", "rstr06_Feb_2022.csv",
                  "rstr07_Mar_2022.csv", "rstr08_Apr_2022.csv", "rstr09_May_2022.csv",
                  "rstr10_Jun_2022.csv", "rstr11_Jul_2022.csv", "rstr12_Aug_2022.csv"]
    return rstr_files

def get_sect_files():
    rstr_files = ["sect01_Sep_2021.csv", "sect02_Oct_2021.csv", "sect03_Nov_2021.csv",
                  "sect04_Dec_2021.csv", "sect05_Jan_2022.csv", "sect06_Feb_2022.csv",
                  "sect07_Mar_2022.csv", "sect08_Apr_2022.csv", "sect09_May_2022.csv",
                  "sect10_Jun_2022.csv", "sect11_Jul_2022.csv", "sect12_Aug_2022.csv"]
    return rstr_files

def get_verbose():
    verbose = "Yes" # "Yes" or "No"
    return verbose

In [4]:
# get data

def get_providers():
    beds_files              = get_beds_files()
    names_with_some_beds    = []
    for this_file in beds_files:
        df          = pd.read_csv(this_file, header=None)
        df.columns  = ["Month","Provider","Metric","Count"]
        for index, row in df.iterrows():
            if row["Count"] != "*":
                names_with_some_beds.append(row["Provider"])
    providers   = sorted(list(set(names_with_some_beds)))
    return providers

In [5]:
# build tables

def build_monthly_providers_beds(providers):
    beds_files              = get_beds_files()
    months                  = [x[0:10] for x in beds_files]
    monthly_providers_beds  = pd.DataFrame(columns = months, index = providers)
    for this_file in beds_files:
        month                       = this_file[0:10]
        monthly_bed_info            = pd.read_csv(this_file, header=None)
        monthly_bed_info.columns    = ["Month","Provider","Metric","Count"]
        for index, row in monthly_bed_info.iterrows():
            if row["Provider"] in providers:
                monthly_providers_beds.at[row["Provider"], month] = row["Count"]
                
    monthly_providers_beds.replace("*",   "0", inplace=True)
    monthly_providers_beds.fillna("0", inplace=True)
    return monthly_providers_beds

def build_monthly_providers_rstr(providers):
    rstr_files              = get_rstr_files()
    months                  = [x[0:10] for x in rstr_files]
    monthly_providers_rstr  = pd.DataFrame(columns = months, index = providers)
    for this_file in rstr_files:
        month                       = this_file[0:10]
        monthly_rstr_info            = pd.read_csv(this_file, header=None)
        monthly_rstr_info.columns    = ["Month","Provider","Metric","Count"]
        for index, row in monthly_rstr_info.iterrows():
            if row["Provider"] in providers:
                monthly_providers_rstr.at[row["Provider"], month] = row["Count"]
    monthly_providers_rstr.replace("*",   "0", inplace=True)
    monthly_providers_rstr.fillna("0", inplace=True)
    return monthly_providers_rstr

def build_monthly_providers_sect(providers):
    sect_files              = get_sect_files()
    months                  = [x[0:10] for x in sect_files]
    monthly_providers_sect  = pd.DataFrame(columns = months, index = providers)
    for this_file in sect_files:
        month                       = this_file[0:10]
        monthly_sect_info            = pd.read_csv(this_file, header=None)
        monthly_sect_info.columns    = ["Month","Provider","Metric","Count"]
        for index, row in monthly_sect_info.iterrows():
            if row["Provider"] in providers:
                monthly_providers_sect.at[row["Provider"], month] = row["Count"]
    monthly_providers_sect.replace("*",   "0", inplace=True)
    monthly_providers_sect.fillna("0", inplace=True)
    return monthly_providers_sect  

In [6]:
# maths model

def order_providers_by_size(monthly_providers_beds):
        # returns NaN for all 0 but none should have that
        # no test - fell out of writing a higher function
        providers_by_size               = monthly_providers_beds.copy()
        providers_by_size               = providers_by_size.astype(int)
        providers_by_size["ProperMean"] = providers_by_size.mask(providers_by_size==0).mean(axis=1)
        providers_by_size               = providers_by_size.sort_values(by="ProperMean")
        providers_by_size               = providers_by_size.sort_values(by="ProperMean")
        providers_by_size.drop(["beds01_Sep","beds02_Oct","beds03_Nov",
                                "beds04_Dec","beds05_Jan","beds06_Feb",
                                "beds07_Mar","beds08_Apr","beds09_May",
                                "beds10_Jun","beds11_Jul","beds12_Aug",
                                "ProperMean"], axis=1, inplace=True)
        return providers_by_size

    

In [7]:
def laplace_correction_and_log(main_table):
    main_table  = main_table.astype(int)
    log_main    = main_table.transform(lambda x: np.log2(x+1))
    return log_main

In [13]:
# view

def greet():
    print("""Welcome to PROD ALERT 2.
             These results are not official do not read them.
             They have not yet been peer reviewed.
             This work is in development.
             The point is that the development will have been done openly, in retrospect, later.
             The underlying figures are already available at NHS Digital.""")
    
def maybe_tell_providers(providers_by_size):
    verbose = get_verbose()
    if verbose == "Yes":
        print("\nThese providers had some beds here they are in size of reported beds:\n")
        display(providers_by_size)

def maybe_tell_log_main(log_main):
    verbose = get_verbose()
    if verbose == "Yes":
        print("\nSize-ordered providers with reported bed-days, sectioned people and restrained people, transformed for scale:\n")
        display(log_main)

def maybe_tell_main_table(main_table):
    verbose = get_verbose()
    if verbose == "Yes":
        print("\nSize-ordered providers with reported bed-days, sectioned people and restrained people:\n")
        display(main_table)

def maybe_tell_monthly_providers_beds(monthly_providers_beds):
    verbose = get_verbose()
    if verbose == "Yes":
        print("""\nAlphabetic table of monthly providers and beds
                   * and empty are replaced with 0:\n""")
        display(monthly_providers_beds)
        
def maybe_tell_monthly_providers_rstr(monthly_providers_rstr):
    verbose = get_verbose()
    if verbose == "Yes":
        print("""\nAlphabetic table of monthly providers and reported people restrained per month
                   * and empty are replaced with 0:\n""")
        display(monthly_providers_rstr)
        
def maybe_tell_monthly_providers_sect(monthly_providers_sect):
    verbose = get_verbose()
    if verbose == "Yes":
        print("""\nAlphabetic table of monthly providers and reported people detained per month
                   * and empty are replaced with 0:\n""")
        display(monthly_providers_sect)

In [14]:
# control

if __name__=="__main__":
                
    greet()
                            
    providers               = get_providers()
    # exclude childrens hospitals with no detentions
    
    monthly_providers_beds  = build_monthly_providers_beds(providers)
    monthly_providers_rstr  = build_monthly_providers_rstr(providers)
    monthly_providers_sect  = build_monthly_providers_sect(providers)
    ordered_providers       = order_providers_by_size(monthly_providers_beds)
    
    beds = pd.concat([ordered_providers,monthly_providers_beds], axis=1).reindex(ordered_providers.index)
    sect = pd.concat([beds,monthly_providers_sect], axis=1).reindex(beds.index)
    rstr = pd.concat([sect,monthly_providers_rstr], axis=1).reindex(sect.index)
                
    main_table  = rstr.copy()
    log_main    = laplace_correction_and_log(main_table)
    
    maybe_tell_providers(ordered_providers)
    maybe_tell_monthly_providers_beds(monthly_providers_beds)
    maybe_tell_monthly_providers_sect(monthly_providers_sect)
    maybe_tell_monthly_providers_rstr(monthly_providers_rstr)
    maybe_tell_main_table(main_table)
    
    maybe_tell_log_main(log_main)
    
    
    
    


Welcome to PROD ALERT 2.
             These results are not official do not read them.
             They have not yet been peer reviewed.
             This work is in development.
             The point is that the development will have been done openly, in retrospect, later.
             The underlying figures are already available at NHS Digital.

These providers had some beds here they are in size of reported beds:



""
SCHOEN CLINIC YORK
GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
TURNING POINT
NEWMARKET HOUSE HEALTHCARE LTD
WHITTINGTON HEALTH NHS TRUST
MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
NEWBRIDGE CARE SYSTEMS LTD
BRAMLEY HEALTH



Alphabetic table of monthly providers and beds
                   * and empty are replaced with 0:



,beds01_Sep,beds02_Oct,beds03_Nov,beds04_Dec,beds05_Jan,beds06_Feb,beds07_Mar,beds08_Apr,beds09_May,beds10_Jun,beds11_Jul,beds12_Aug
ACTIVE PATHWAYS,0,410,420,415,415,390,995,925,955,890,880,860
ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,215,175,135,135,220,250,280,240,270,240,215,245
AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NHS TRUST,13380,13775,13355,13955,13720,12305,13695,13345,13660,13490,14080,14230
"BARNET, ENFIELD AND HARINGEY MENTAL HEALTH NHS TRUST",15220,15800,15250,15465,15395,14110,15430,15125,15410,14850,15500,15685
BERKSHIRE HEALTHCARE NHS FOUNDATION TRUST,4175,4215,4145,3910,4140,3655,4115,3965,4300,4335,4440,4470
BIRMINGHAM AND SOLIHULL MENTAL HEALTH NHS FOUNDATION TRUST,19685,20295,19640,19950,20035,18210,19875,19315,19875,19500,19835,19875
BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST,600,640,595,615,600,530,570,545,675,635,645,610
BLACK COUNTRY HEALTHCARE NHS FOUNDATION TRUST,8855,9050,8445,9185,9080,8420,9665,8820,9660,8935,9085,9480
BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,4995,5270,5200,5050,4540,4355,4915,4775,5090,5050,5310,5370
BRAMLEY HEALTH,420,0,425,465,385,330,340,330,340,330,340,340



Alphabetic table of monthly providers and reported people detained per month
                   * and empty are replaced with 0:



...

,rstr01_Sep,rstr02_Oct,rstr03_Nov,rstr04_Dec,rstr05_Jan,rstr06_Feb,rstr07_Mar,rstr08_Apr,rstr09_May,rstr10_Jun,rstr11_Jul,rstr12_Aug
ACTIVE PATHWAYS,0,0,0,0,0,0,0,0,0,0,0,0
ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,0,0,0,0,0,0,0,0,0,0,0,0
AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NHS TRUST,60,0,0,0,0,0,55,55,0,60,75,70
"BARNET, ENFIELD AND HARINGEY MENTAL HEALTH NHS TRUST",35,45,45,35,30,10,20,30,35,45,40,45
BERKSHIRE HEALTHCARE NHS FOUNDATION TRUST,30,0,0,0,0,0,0,35,15,25,30,20
BIRMINGHAM AND SOLIHULL MENTAL HEALTH NHS FOUNDATION TRUST,105,95,80,70,70,80,80,75,95,85,75,100
BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST,0,0,0,0,0,0,0,0,0,0,0,0
BLACK COUNTRY HEALTHCARE NHS FOUNDATION TRUST,0,65,55,45,55,60,65,50,60,55,55,0
BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,0,0,0,0,0,0,0,0,0,0,0,0
BRAMLEY HEALTH,0,0,0,0,0,0,0,0,0,0,0,0



Size-ordered providers with reported bed-days, sectioned people and restrained people:



,beds01_Sep,beds02_Oct,beds03_Nov,beds04_Dec,beds05_Jan,beds06_Feb,beds07_Mar,beds08_Apr,beds09_May,beds10_Jun,beds11_Jul,beds12_Aug,sect01_Sep,sect02_Oct,sect03_Nov,sect04_Dec,sect05_Jan,sect06_Feb,sect07_Mar,sect08_Apr,sect09_May,sect10_Jun,sect11_Jul,sect12_Aug,rstr01_Sep,rstr02_Oct,rstr03_Nov,rstr04_Dec,rstr05_Jan,rstr06_Feb,rstr07_Mar,rstr08_Apr,rstr09_May,rstr10_Jun,rstr11_Jul,rstr12_Aug
SCHOEN CLINIC YORK,0,390,0,0,0,0,55,160,185,140,95,125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST,190,195,195,215,215,195,200,160,155,180,185,185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,215,175,135,135,220,250,280,240,270,240,215,245,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TURNING POINT,0,0,0,0,0,0,0,0,0,490,145,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
NEWMARKET HOUSE HEALTHCARE LTD,270,305,295,285,280,245,305,300,285,300,305,305,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WHITTINGTON HEALTH NHS TRUST,190,280,320,320,345,315,305,305,310,310,325,305,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,260,250,325,0,0,305,350,295,290,270,325,355,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST,120,125,120,270,240,255,320,405,500,500,585,570,0,0,0,5,10,10,10,5,15,15,15,15,0,0,0,0,0,0,0,0,0,0,0,0
NEWBRIDGE CARE SYSTEMS LTD,365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BRAMLEY HEALTH,420,0,425,465,385,330,340,330,340,330,340,340,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Size-ordered providers with reported bed-days, sectioned people and restrained people, transformed for scale:



,beds01_Sep,beds02_Oct,beds03_Nov,beds04_Dec,beds05_Jan,beds06_Feb,beds07_Mar,beds08_Apr,beds09_May,beds10_Jun,beds11_Jul,beds12_Aug,sect01_Sep,sect02_Oct,sect03_Nov,sect04_Dec,sect05_Jan,sect06_Feb,sect07_Mar,sect08_Apr,sect09_May,sect10_Jun,sect11_Jul,sect12_Aug,rstr01_Sep,rstr02_Oct,rstr03_Nov,rstr04_Dec,rstr05_Jan,rstr06_Feb,rstr07_Mar,rstr08_Apr,rstr09_May,rstr10_Jun,rstr11_Jul,rstr12_Aug
SCHOEN CLINIC YORK,0.000000,8.611025,0.000000,0.000000,0.000000,0.000000,5.807355,7.330917,7.539159,7.139551,6.584963,6.977280,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST,7.577429,7.614710,7.614710,7.754888,7.754888,7.614710,7.651052,7.330917,7.285402,7.499846,7.539159,7.539159,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,7.754888,7.459432,7.087463,7.087463,7.787903,7.971544,8.134426,7.912889,8.082149,7.912889,7.754888,7.942515,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TURNING POINT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.939579,7.189825,7.189825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NEWMARKET HOUSE HEALTHCARE LTD,8.082149,8.257388,8.209453,8.159871,8.134426,7.942515,8.257388,8.233620,8.159871,8.233620,8.257388,8.257388,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
WHITTINGTON HEALTH NHS TRUST,7.577429,8.134426,8.326429,8.326429,8.434628,8.303781,8.257388,8.257388,8.280771,8.280771,8.348728,8.257388,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,8.027906,7.971544,8.348728,0.000000,0.000000,8.257388,8.455327,8.209453,8.184875,8.082149,8.348728,8.475733,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST,6.918863,6.977280,6.918863,8.082149,7.912889,8.000000,8.326429,8.665336,8.968667,8.968667,9.194757,9.157347,0.000000,0.000000,0.000000,2.584963,3.459432,3.459432,3.459432,2.584963,4.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NEWBRIDGE CARE SYSTEMS LTD,8.515700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BRAMLEY HEALTH,8.717676,0.000000,8.734710,8.864186,8.592457,8.370687,8.413628,8.370687,8.413628,8.370687,8.413628,8.413628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
# tests

def test_build_monthly_providers_beds():
    providers              = get_providers()
    monthly_providers_beds = build_monthly_providers_beds(providers)
    assert type(monthly_providers_beds)    == pd.DataFrame, "type test"
    assert type(monthly_providers_beds["beds01_Sep"]) == pd.Series, "type test"
    assert monthly_providers_beds.at["WEST LONDON NHS TRUST","beds01_Sep"] == "19235", "correct value and type"
    assert monthly_providers_beds.at["WEST LONDON NHS TRUST","beds01_Sep"] != 19235,   "correct value and type"
    assert monthly_providers_beds.at["WEST LONDON NHS TRUST","beds01_Sep"] != "999",   "correct value and type"
    assert len(monthly_providers_beds)     == len(set(providers)), "uniqueness test"
    assert monthly_providers_beds.index[1] == providers[1], "matches providers test"
    print("passed build monthly providers beds")
    
def test_build_monthly_providers_rstr():
    providers              = get_providers()
    monthly_providers_rstr = build_monthly_providers_rstr(providers)
    assert type(monthly_providers_rstr)    == pd.DataFrame, "type test"
    assert type(monthly_providers_rstr["rstr01_Sep"]) == pd.Series, "type test"
    assert monthly_providers_rstr.at["WEST LONDON NHS TRUST","rstr01_Sep"] == "75", "correct value and type"
    assert monthly_providers_rstr.at["WEST LONDON NHS TRUST","rstr01_Sep"] != 75,   "correct value and type"
    assert monthly_providers_rstr.at["WEST LONDON NHS TRUST","rstr01_Sep"] != "999","correct value and type"  
    assert len(monthly_providers_rstr)     == len(set(providers)), "uniqueness test"
    assert monthly_providers_rstr.index[1] == providers[1], "matches providers test"
    print("passed build monthly providers rstr")
    
def test_build_monthly_providers_sect():
    providers              = get_providers()
    monthly_providers_sect = build_monthly_providers_sect(providers)
    assert type(monthly_providers_sect)    == pd.DataFrame, "type test"
    assert type(monthly_providers_sect["sect01_Sep"]) == pd.Series, "type test"
    assert monthly_providers_sect.at["WEST LONDON NHS TRUST","sect01_Sep"] == "565", "correct value and type"
    assert monthly_providers_sect.at["WEST LONDON NHS TRUST","sect01_Sep"] != 565,   "correct value and type"
    assert monthly_providers_sect.at["WEST LONDON NHS TRUST","sect01_Sep"] != "999","correct value and type"  
    assert len(monthly_providers_sect)     == len(set(providers)), "uniqueness test"
    assert monthly_providers_sect.index[1] == providers[1], "matches providers test"
    print("passed build monthly providers sect")
    
def test_get_providers():
    providers = get_providers()
    assert type(providers)    == list, "type test"
    assert type(providers[1]) == str, "type test"
    assert len(providers)  == len(set(providers)), "uniqueness test"
    print("passed get providers")
    
def test_laplace_correction_and_log():
    toy         = {'Number':[4,3,2,1]}
    toy_df      = pd.DataFrame(toy)
    log_main    = laplace_correction_and_log(toy_df)
    assert list(log_main.Number) == [2.321928094887362, 2.0, 1.584962500721156, 1.0]    
    toy         = {'Jam':[4,3,2,1],'Spam':[15,31,63,0]}
    toy_df      = pd.DataFrame(toy)
    log_main    = laplace_correction_and_log(toy_df)
    assert list(log_main.Jam)  == [2.321928094887362, 2.0, 1.584962500721156, 1.0]
    assert list(log_main.Spam) == [4.0, 5.0, 6.0, 0.0]
    print("passed laplace correction and log")

def all_tests():
    test_build_monthly_providers_beds()
    test_build_monthly_providers_rstr()
    test_build_monthly_providers_sect()  
    test_get_providers()
    test_laplace_correction_and_log()
    print("passed all tests")
    
all_tests()    

passed build monthly providers beds
passed build monthly providers rstr
passed build monthly providers sect
passed get providers
passed laplace correction and log
passed all tests


In [ ]:
# scraps

#
#def allocate_providers(providers_by_size):
    
    # how small is small
    #     it is small enough to be
    #         in the continuous series of smallest institutions
    #         with “null” reports commensurate with size for sect and rstr
    #         having other monthly reports around 5 and 10
    
    # start at smallest work up by size through sect and rstr checking continuity
    #     and that other values are 5 or 10
    
    # are you big or small
    #     if you are small we believe your zeros and use them in the average
    #     if you are big we do not believe your zeros and we only average your big figures
    
    # if you are big and have n>=11 months of the year count>0 you are a complete reporter
    #     we replace your only 0 with the last value
    
    # if you are big and have on-off reporting you are partial and we do the estimate based
    #     we only average your big figures
    
    # if you are big and join in reporting
    #     we only average your later consecutive non-zero reports
    
#    allocation = {"foo":"bar"}
#    return allocation

#def test_allocate_providers():
#    providers               = get_providers()
#    monthly_providers_beds  = build_monthly_providers_beds(providers)
#    allocation              = allocate_providers(monthly_providers_beds)
#    assert type(allocation) == dict, "type check"
#    
#test_allocate_providers()
